In [1]:
import numpy as np
import time
import math
from copy import deepcopy
from tomography import *
from NestedForLoop import get_iterator
from pathlib import Path
import os
import glob
import pandas as pd
import fnmatch
from efficiencies import finding_file, get_channels_eff, set_raw_counts
from optimization import Optimizer, function_fidelity, FidelityResults
from constants import *
from densitymatrix import DensityMatrix, apply_unitary_to_dm, GHZ
from pathlib import Path
import fnmatch
from pauliexpectations import *
import itertools

In [2]:
working_dir = os.getcwd()
working_dir_data = os.getcwd()+'\StateTomoData'
working_dir=r"C:\Users\nicol\Desktop\Experiment"
os.chdir(working_dir)

In [3]:
######################################################
#----- CREATING THE FILES FOR AND THE COUNTS -------
######################################################


print('Creating the counts and files')
qubit_number=4
bell=(np.array([1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])+np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]))/np.sqrt(2)
#bell=(np.array([0,1,0,0])+np.array([0,0,1,0]))/np.sqrt(2)
Counts_file = np.zeros((3**qubit_number,2**qubit_number))
letters = ['x', 'y', 'z']
bases = []

def get_proj_iterator(N_qubit):
    d={"x": [[1/np.sqrt(2),1/np.sqrt(2)],[1/np.sqrt(2),-1/np.sqrt(2)]], "y":[[1/np.sqrt(2),complex(0,1/np.sqrt(2))],[1/np.sqrt(2),complex(0,-1/np.sqrt(2))]], "z":[[1,0],[0,1]] }
    basis_order= get_iterator(3,N_qubit)
    basis=["x","y","z"]
    projectors_order=get_iterator(2,N_qubit)
    columns=[]
    
    
    character_Matrix=[]
    for i in basis_order:
        string=""
        for j in i:
            string=string+basis[j]
        lines=[]    
        for k in projectors_order:
            o=0
            vect=[]
            for n in k:
                if o > 0:
                    vect=np.kron(vect,d[string[o]][n])
                else:
                    vect=d[string[o]][n]
                o=o+1
            lines.append(vect)    
        character_Matrix.append(lines)         
        columns.append(string)   
        
    return character_Matrix 
Projector_matrix=  get_proj_iterator(qubit_number)

for i in range(3**qubit_number):
    for j in range(2**qubit_number):
        Counts_file[i][j] = 100000*abs(np.trace(np.dot(np.outer(Projector_matrix[i][j],Projector_matrix[i][j]),np.outer(bell,bell))))  

for i in letters:
    if qubit_number == 1:
        base = i 
        bases.append(base)
    for j in letters:
        if qubit_number == 2:
            base = i + j
            bases.append(base)
        for k in letters:
            if qubit_number == 3:
                base = i + j + k 
                bases.append(base)
            for l in letters:
                if qubit_number == 4:
                    base = i + j + k + l
                    bases.append(base)

                    
for i in range(len(bases)):
    f = open(r"C:/Users/nicol/Desktop/Experiment/StateTomography_2Layers_20230330/StateTomo_2Layers/Bigiteration_0_" + str(bases[i]) + ".txt", "a")    
    f.truncate(0)
    print(bases[i])
    for j in range(2**qubit_number):
        print(Counts_file[i,j])
        f.write(str(Counts_file[i,j]))
        f.write(' ')
    f.close()


Creating the counts and files
xxxx
12499.999999999989
0.0
0.0
12499.999999999989
0.0
12499.999999999989
12499.999999999989
0.0
0.0
12499.999999999989
12499.999999999989
0.0
12499.999999999989
0.0
0.0
12499.999999999989
xxxy
6249.9999999999945
6249.9999999999945
6249.9999999999945
6249.9999999999945
6249.9999999999945
6249.9999999999945
6249.9999999999945
6249.9999999999945
6249.9999999999945
6249.9999999999945
6249.9999999999945
6249.9999999999945
6249.9999999999945
6249.9999999999945
6249.9999999999945
6249.9999999999945
xxxz
6249.999999999995
6249.999999999995
6249.999999999995
6249.999999999995
6249.999999999995
6249.999999999995
6249.999999999995
6249.999999999995
6249.999999999995
6249.999999999995
6249.999999999995
6249.999999999995
6249.999999999995
6249.999999999995
6249.999999999995
6249.999999999995
xxyx
6249.9999999999945
6249.9999999999945
6249.9999999999945
6249.9999999999945
6249.9999999999945
6249.9999999999945
6249.9999999999945
6249.9999999999945
6249.9999999999945
624

In [4]:
print(bases)

['xxxx', 'xxxy', 'xxxz', 'xxyx', 'xxyy', 'xxyz', 'xxzx', 'xxzy', 'xxzz', 'xyxx', 'xyxy', 'xyxz', 'xyyx', 'xyyy', 'xyyz', 'xyzx', 'xyzy', 'xyzz', 'xzxx', 'xzxy', 'xzxz', 'xzyx', 'xzyy', 'xzyz', 'xzzx', 'xzzy', 'xzzz', 'yxxx', 'yxxy', 'yxxz', 'yxyx', 'yxyy', 'yxyz', 'yxzx', 'yxzy', 'yxzz', 'yyxx', 'yyxy', 'yyxz', 'yyyx', 'yyyy', 'yyyz', 'yyzx', 'yyzy', 'yyzz', 'yzxx', 'yzxy', 'yzxz', 'yzyx', 'yzyy', 'yzyz', 'yzzx', 'yzzy', 'yzzz', 'zxxx', 'zxxy', 'zxxz', 'zxyx', 'zxyy', 'zxyz', 'zxzx', 'zxzy', 'zxzz', 'zyxx', 'zyxy', 'zyxz', 'zyyx', 'zyyy', 'zyyz', 'zyzx', 'zyzy', 'zyzz', 'zzxx', 'zzxy', 'zzxz', 'zzyx', 'zzyy', 'zzyz', 'zzzx', 'zzzy', 'zzzz']


In [5]:
######################################################
#----- CREATING THE FILES FOR THE EFFICIENCIES -------
######################################################



print('Creating the counts and files for the efficiency')
letters = ['z', 'a']
bases = []

def get_proj_iterator(N_qubit):
    d={"z":[[1,0],[0,1]], "a":[[0,1],[1,0]]}
    basis_order= get_iterator(2,N_qubit)
    basis=["z","a"]
    projectors_order=get_iterator(2,N_qubit)
    columns=[]
    
    
    character_Matrix=[]
    for i in basis_order:
        string=""
        for j in i:
            string=string+basis[j]
        lines=[]    
        for k in projectors_order:
            o=0
            vect=[]
            for n in k:
                if o > 0:
                    vect=np.kron(vect,d[string[o]][n])
                else:
                    vect=d[string[o]][n]
                o=o+1
            lines.append(vect)    
        character_Matrix.append(lines)         
        columns.append(string)   
        
    return character_Matrix 
Projector_matrix2=  get_proj_iterator(qubit_number)

for i in range(2**qubit_number):
    for j in range(2**qubit_number):
        Counts_file[i][j] = 1000*abs(np.trace(np.dot(np.outer(Projector_matrix2[i][j],Projector_matrix2[i][j]),np.outer(bell,bell))))  

for i in letters:
    if qubit_number == 1:
        base = i 
        bases.append(base)
    for j in letters:
        if qubit_number == 2:
            base = i + j
            bases.append(base)
        for k in letters:
            if qubit_number == 3:
                base = i + j + k 
                bases.append(base)
            for l in letters:
                if qubit_number == 4:
                    base = i + j + k + l
                    bases.append(base)
                    
for i in range(len(bases)):
    f = open(r"C:/Users/nicol/Desktop/Experiment/StateTomography_2Layers_20230330/StateTomo_2Layers/Bigiteration_0_" + str(bases[i]) + ".txt", "a")    
    f.truncate(0)
    for j in range(2**qubit_number):
        f.write(str(Counts_file[i,j]))
        f.write(' ')
    f.close()


Creating the counts and files for the efficiency


In [6]:
######################################################################################################
#----- COUNTING THE FILES AND SAVING THEM IN AN ARRAY TO MAKES THE REST OF THE ANALYSIS EASIER -------
######################################################################################################


print('COUNTING THE FILES AND SAVING THEM IN AN ARRAY TO MAKES THE REST OF THE ANALYSIS EASIER')

n_files=0
working_dir=r"C:\Users\nicol\Desktop\Experiment"
os.chdir(working_dir)


filenames = [i for i in glob.glob("StateTomography_2Layers_*")]
filenames.sort(key=os.path.getmtime)

index_to_file = {}

for index, filename in enumerate(filenames):
    os.chdir(f"{working_dir}\\{filename}")
    filenames_aux=[i for i in glob.glob("StateTomo_2Layers*")]
    for index_second, filenames_aux_second in enumerate(filenames_aux):
        index_to_file[n_files] = f"{filename}\\{filenames_aux_second}"
        n_files+=1
os.chdir(working_dir)

COUNTING THE FILES AND SAVING THEM IN AN ARRAY TO MAKES THE REST OF THE ANALYSIS EASIER


In [7]:
os.chdir(working_dir)
qubit_number=4

## Defining the columns of the data file we want to use as data to reconstruct the density matrix (eg.: HH HV VH and VV basis)
column_start=0
column_stop=16

##column_start_2_emissions=0
##column_stop_2_emissions=16

state_after=[]
state_after_file=[]

state_before=[]
state_before_file=[]

state = []
state_file=[]

xp_counts_corrected_with_eff=[]

statetomo = []

#####################################################################
#---------------------- STATE TOMOGRAPHY ----------------------------
#####################################################################

for index in range(len(index_to_file)):
    os.chdir(f"{working_dir}\\{index_to_file[index]}\\")
    datafiles=[i for i in glob.glob("*")]
                
    ### Calculating the efficiencies of each detector
    print('Calculating the efficiencies')
    efficiencies=get_channels_eff(datafiles,qubit_number, column_start, column_stop, os.getcwd())
    ##efficiencies_2_emissions=get_channels_eff(datafiles,qubit_number, column_start_2_emissions, column_stop_2_emissions, os.getcwd())
 

    ### Opening the data files and writing the data in counts_aux array
    print('Gathering the counts')
    counts_aux=set_raw_counts(datafiles, qubit_number, column_start, column_stop, os.getcwd())
    xp_counts=np.array(np.transpose(counts_aux))
 
    ##counts_aux_2_emissions=set_raw_counts(datafiles, qubit_number, column_start_2_emissions, column_stop_2_emissions, os.getcwd())
    ##xp_counts_2_emissions=np.array(np.transpose(counts_aux_2_emissions))

    print('Performing the tomography')
    statetomo.append(LRETomography(int(qubit_number),xp_counts))
    statetomo[-1].run(correct_eff=efficiencies)#,correct_double_emission = efficiencies_2_emissions)
    xp_counts_corrected_with_eff.append(statetomo[-1].xp_counts)
    state.append(statetomo[-1])#.state)
    state_file.append(index_to_file[index])
    
    

Calculating the efficiencies
Gathering the counts
Performing the tomography


In [8]:
######################################
#-- DEFINING THE TARGET BELL STATE ---
######################################

qubit_number = 4
# bell=(np.array([1,0,0,0])-np.array([0,0,0,1]))/np.sqrt(2)
#bell=(np.array([1,0,0,0,0,0,0,0])+np.array([0,0,0,0,0,0,0,1]))/np.sqrt(2)
bell=(np.array([1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])+np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]))/np.sqrt(2)
# bell=(np.array([0,1,0,0])-np.array([0,0,1,0]))/np.sqrt(2)
#bell=(np.array([0,1,0,0])+np.array([0,0,1,0]))/np.sqrt(2)
bellmatrix=np.array(np.outer(bell, np.conjugate(bell)))


##########################################################
#----- OPTIMIZATION OF MAX FIDELITY UP TO UNITARIES ------
##########################################################
states=state
fid=np.zeros((n_files))
optimized_matrix=np.zeros((n_files,2**qubit_number,2**qubit_number), dtype='complex')

guess=np.array([0, 0, 0, 0, 0, 0, 0,0,0])
bounds=[(-np.pi,np.pi)]*9
results = []

print('Getting the unitary')
opt=Optimizer(guess,function_fidelity, results=FidelityResults)

for index in range(len(states)):
    result=opt.optimize(qubit_number,states[index].state, bell,bounds=bounds)
    results.append(result)

Getting the unitary


In [14]:
##########################################################
#------------ ERRORS Input with BELL STATE---------------#
##########################################################
error_runs=10

U=[]
bell_aux=[]
target_ini=[]

states=state
states_file=state_file
players=["Arya","Bran","Cersei","Dany"]


for index in range(len(states)):
    target=bellmatrix
    U.append(results[index].correct(qubit_number))
    target_ini.append(np.transpose(np.conjugate(U[-1]))@bellmatrix@U[-1])    
    
    states[index].calculate_fidelity_error(players, error_runs, opt, target, optimization=True, bounds=bounds)
    
    print('file, fidelity, fidelity_mean, fidelity_std: ',
          states_file[index], np.round(states[index].state.fidelity(target_ini[-1]),5), np.round(states[index].fidelity_mu,5),np.round(states[index].fidelity_std,5), '\n')

Simulating new states considering the uncertainties
Optimizing the fidelity between input and target up to a unitary
file, fidelity, fidelity_mean, fidelity_std:  StateTomography_2Layers_20230330\StateTomo_2Layers (1+0j) -0.99737 0.00039 

